## Imports and xml wrapper for WPS request

In [1]:
import requests
from osgeo import ogr

def wrapXML(proces,geom):
    xml_head='<?xml version="1.0" encoding="UTF-8"?><wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd"><ows:Identifier>%s</ows:Identifier>'%proces
    xml_tail = '<wps:ResponseForm><wps:RawDataOutput mimeType="application/json"><ows:Identifier>result</ows:Identifier></wps:RawDataOutput></wps:ResponseForm></wps:Execute>'
    xml_geom='<wps:DataInputs>'
    for identifier,geometria in geom['geoms'].items():
        xml_geom+='<wps:Input><ows:Identifier>%s</ows:Identifier><wps:Data><wps:ComplexData mimeType="application/json"><![CDATA[%s]]></wps:ComplexData></wps:Data></wps:Input>'%(identifier,geometria)
    if 'literals' in geom.keys():
        for identifier,geometria in geom['literals'].items():
            xml_geom+='<wps:Input><ows:Identifier>%s</ows:Identifier><wps:Data><wps:LiteralData>%s</wps:LiteralData></wps:Data></wps:Input>'%(identifier,geometria)
    xml_geom+='</wps:DataInputs>'
    xml=xml_head.strip()+xml_geom.strip()+xml_tail.strip()
    return xml.strip()

## Preparing input geometries and other parameters

In [10]:
# polygon1 = [(-102.8184, 39.5273), (-102.8184, 37.418), (-101.2363, 37.418), (-101.2363, 39.5273), (-102.8184, 39.5273)]
polygon1 = [( 18.,52.5), (21.,52.5), (21.2, 53.5), (18.,53.5),( 18.,52.5)]
polygon2 = [(-92.8184, 39.5273), (-92.8184, 37.418), (-91.2363, 37.418), (-91.2363, 39.5273), (-92.8184, 39.5273)] 
poly1 = ogr.Geometry(ogr.wkbLinearRing)
poly2 = ogr.Geometry(ogr.wkbLinearRing)
for i in polygon1:
    poly1.AddPoint(*i)

for i in polygon2:
    poly2.AddPoint(*i)

p=poly1.ExportToJson()
p1=poly2.ExportToJson()

## Request for distance between two geometries

In [11]:
g={'geoms':{'a':p,'b':p1}}
r = requests.post("http://185.52.194.216:8080/geoserver/wps", data=wrapXML("JTS:distance",g))
print("Connection status:")
print(r.status_code, r.reason)
print("WPS response:")
print(r.text[:] + '...')

Connection status:
200 OK
WPS response:
110.00390985315022...


## Request for geometry buffer zone

In [12]:
g={'geoms':{'geom':p},'literals':{'distance':'0.01','quadrantSegments':'1.11','capStyle':'Round'}}
r = requests.post("http://185.52.194.216:8080/geoserver/wps", data=wrapXML("JTS:buffer",g))
print("Connection status:")
print(r.status_code, r.reason)
print("WPS response:")
print(r.text[:] + '...')
odpowiedz = r.text[:]

Connection status:
200 OK
WPS response:
{"type":"Polygon","coordinates":[[[17.99,52.5],[17.99,53.5],[18,53.51],[21.2,53.51],[21.2098,53.498],[21.0098,52.498],[21,52.49],[18,52.49],[17.99,52.5]],[[18.01,52.51],[20.9918,52.51],[21.1878,53.49],[18.01,53.49],[18.01,52.51]]]}...


In [17]:
import json
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
dc = DrawControl(circle={},rectangle={},polyline={},polygon={}
                ,marker={'shapeOptions':{'color':'#5300FF'}},edit=False)
layers=[]
j=json.loads(odpowiedz)
print (j)
g=GeoJSON(data=j,hover_style={'fillColor': 'red'})
layers.append(g)
m=Map(center=(53,21))
m.add_layer(g)
m

{'type': 'Polygon', 'coordinates': [[[17.9, 52.5], [17.9, 53.5], [18, 53.6], [21.2, 53.6], [21.2981, 53.4804], [21.0981, 52.4804], [21, 52.4], [18, 52.4], [17.9, 52.5]], [[18.1, 52.6], [20.918, 52.6], [21.078, 53.4], [18.1, 53.4], [18.1, 52.6]]]}


In [ ]:
g=GeoJSON(data=h,hover_style={'fillColor': 'red'})
    layers.append(g)